In [9]:
import torch
import math
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

# torch.manual_seed(1)    # reproducible

# Hyper Parameters
TIME_STEP = 26      # rnn time step
INPUT_SIZE = 26      # rnn input size
LR = 0.0001           # learning rate

# show data
# steps = np.linspace(0, np.pi*2, 100, dtype=np.float32)  # float32 for converting torch FloatTensor
# x_np = np.sin(steps)
# y_np = np.cos(steps)
# plt.plot(steps, y_np, 'r-', label='target (cos)')
# plt.plot(steps, x_np, 'b-', label='input (sin)')
# plt.legend(loc='best')
# plt.show()

pre = [[0 for j in range(0,26)] for i in range(0,26)]
x = []
y = [(i+1)%26 for i in range(0,26)]
for i in range(0,26):
    x.append([])
    for j in range(0,26):
        x[i].append(0)
        if i == j:
            x[i][j] = 1

x = torch.from_numpy(np.array(x).reshape((-1,TIME_STEP,INPUT_SIZE))).float()
y = torch.from_numpy(np.array(y).reshape((-1,INPUT_SIZE))).long()
# print(x)

# print(x.shape, "\n", y.shape)
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size=INPUT_SIZE,
            hidden_size=128,     # rnn hidden unit
            num_layers=3,       # number of rnn layer
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(128, 26)

    def forward(self, x, h_state):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, output_size)
        r_out, h_state = self.rnn(x, h_state)

        outs = []    # save all predictions
        for time_step in range(r_out.size(1)):    # calculate output for each time step
            outs.append(self.out(r_out[:, time_step, :]))
        return torch.stack(outs, dim=1), h_state

        # instead, for simplicity, you can replace above codes by follows
        # r_out = r_out.view(-1, 32)
        # outs = self.out(r_out)
        # outs = outs.view(-1, TIME_STEP, 1)
        # return outs, h_state
        
        # or even simpler, since nn.Linear can accept inputs of any dimension 
        # and returns outputs with same dimension except for the last
        # outs = self.out(r_out)
        # return outs

rnn = RNN()
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()  

h_state = None      # for initial hidden state

# plt.figure(1, figsize=(12, 5))
# plt.ion()           # continuously plot

for i in range(4000):
    prediction,h_state = rnn(x, h_state)
    h_state = h_state.data
#     print(prediction.size())
    loss = loss_func(prediction[0], y[0])
    optimizer.zero_grad()                   # clear gradients for this training step
    loss.backward()                         # backpropagation, compute gradients
    optimizer.step()                        # apply gradients
    if(i % 200 == 0):
        prediction_index = torch.argmax(prediction,2)
#         print(prediction_index)
        accuracy = float((prediction_index == y).sum())/26
        print("_________________________________________________________________\n")
        print("loss:", loss.data.numpy(), "   accuracy:", accuracy, "\n")




RNN(
  (rnn): RNN(26, 128, num_layers=3, batch_first=True)
  (out): Linear(in_features=128, out_features=26, bias=True)
)
_________________________________________________________________

loss: 3.2695363    accuracy: 0.038461538461538464 

_________________________________________________________________

loss: 0.42689097    accuracy: 1.0 

_________________________________________________________________

loss: 0.08628748    accuracy: 1.0 

_________________________________________________________________

loss: 0.038664218    accuracy: 1.0 

_________________________________________________________________

loss: 0.02256941    accuracy: 1.0 

_________________________________________________________________

loss: 0.0149557255    accuracy: 1.0 

_________________________________________________________________

loss: 0.010681391    accuracy: 1.0 

_________________________________________________________________

loss: 0.008019231    accuracy: 1.0 

_________________________________